In [2]:
!pip install torch transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import os

In [13]:
# Desactivar Weights & Biases de forma segura
os.environ["WANDB_MODE"] = "offline"

In [14]:
# Cargar modelo pre-entrenado (GPT-2 como base)
MODEL_NAME = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # Solución al error de padding
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


In [15]:
# Definir datos de entrenamiento
dialogues = [
    {"input": "Hola, ¿cómo estás?", "output": "¡Hola! Estoy bien, gracias. ¿Y tú?"},
    {"input": "¿Cuál es tu nombre?", "output": "Soy un chatbot basado en Transformers."},
    {"input": "Cuéntame un chiste", "output": "¿Por qué el libro de matemáticas estaba triste? Porque tenía demasiados problemas."}
]

def tokenize_function(example):
    return tokenizer(f"<s>{example['input']} {example['output']}</s>", truncation=True, padding='max_length', max_length=128)


In [16]:
# Crear dataset
dataset = Dataset.from_list(dialogues)
tokenized_datasets = dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [17]:
# Configurar entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to=[]  # Desactivar completamente el logging en W&B
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-17-6923e54b825d>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
# Entrenar modelo
trainer.train()


Step,Training Loss


TrainOutput(global_step=6, training_loss=4.32051436106364, metrics={'train_runtime': 70.7318, 'train_samples_per_second': 0.127, 'train_steps_per_second': 0.085, 'total_flos': 587907072000.0, 'train_loss': 4.32051436106364, 'epoch': 3.0})

In [19]:
# Guardar modelo
model.save_pretrained("./chatbot_model")
tokenizer.save_pretrained("./chatbot_model")

('./chatbot_model/tokenizer_config.json',
 './chatbot_model/special_tokens_map.json',
 './chatbot_model/vocab.json',
 './chatbot_model/merges.txt',
 './chatbot_model/added_tokens.json',
 './chatbot_model/tokenizer.json')

In [20]:
# Función de interacción con el chatbot
def chat():
    print("Chatbot listo. Escribe 'salir' para terminar.")
    while True:
        user_input = input("Tú: ")
        if user_input.lower() == "salir":
            break
        inputs = tokenizer(user_input, return_tensors="pt", padding=True)
        outputs = model.generate(**inputs, max_length=50)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Chatbot:", response)

if __name__ == "__main__":
    chat()


Chatbot listo. Escribe 'salir' para terminar.
Tú: salir
